In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from rdkit import Chem

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv('6_good_smiles.csv')

In [4]:
df

,DOI,Date,Journal,Title,Name,measurement_error,measurement_wavelength,measurement_method,normalised_name,raw_value,specifier
0,10.1016/S0022-3093(99)00330-0,7/19/1999,Journal of Non-Crystalline Solids,PHOTOINDUCEDCHANGESINLINEARNONLINEAROPTICALPRO...,As20S60Ge20,0.0,NaN,el_cde_tables,"[['As', 20.0], ['Ge', 20.0], ['S', 60.0]]",2.054,n
1,10.1016/j.fct.2006.05.017,6/7/2006,Food and Chemical Toxicology,GENERATIONFORMALDEHYDEINCIGARETTESOVERVIEWRECE...,Propionaldehyde,0.0,NaN,el_cde_tables,CCC=O,3.74,n
2,10.1016/j.mee.2011.01.031,1/18/2011,Microelectronic Engineering,SINUSOIDALPLASMONICCRYSTALSFORBIODETECTIONSENSORS,PEO,0.0,NaN,el_cde_text,OO,1.47,refractive index
3,10.1016/j.apradiso.2012.04.026,5/2/2012,Applied Radiation and Isotopes,BENCHMARKINGGEANT4FULLSYSTEMSIMULATIONASSOCIAT...,Aluminum,0.0,NaN,el_cde_tables,[AlH2],4.4,Refractive index
4,10.1016/S0924-4247(02)00264-9,9/16/2002,Sensors and Actuators A: Physical,INFLUENCESMATERIALPROPERTIESCERAMICMICROSTEREO...,Alumina,0.0,NaN,el_mylogic,O=[Al]O[Al]=O,1.7,Refractive index
...,...,...,...,...,...,...,...,...,...,...,...
4658,10.1016/j.molliq.2011.07.002,7/22/2011,Journal of Molecular Liquids,UVSPECTRALCHANGESFORAZOCOMPOUNDSINPRESENCEDIFF...,iso-Butanol,0.0,NaN,el_cde_tables,CC(C)CO,1.3959,n
4659,10.1016/j.asr.2009.06.015,6/26/2009,Advances in Space Research,DIFFERENTAPPROACHESRAYLEIGHOPTICALDEPTHDETERMI...,CO2,0.0,NaN,el_cde_tables,O=C=O,1.000274585,Refractive index
4660,10.1016/S0022-3093(99)00681-X,2/25/2000,Journal of Non-Crystalline Solids,UVTRANSMISSIONRADIATIONINDUCEDDEFECTSINPHOSPHA...,Zn(PO3)2,0.0,NaN,el_cde_tables,CC(=O)OCC1=C2C(=O)O[Zn]OC(=O)C(N)CCCC(=O)N[C@@...,1.526,Refractive index
4661,10.1016/j.jct.2016.04.016,4/20/2016,The Journal of Chemical Thermodynamics,ASYMMETRICCRITICALITYIONICSOLUTIONCONTAINING1H...,2-propanol,0.0,632.8 nm,el_mylogic,CC(C)O,1.3884,n


#### Переведем raw value в числовой формат

In [5]:
# зададим функцию для очискри raw value
def clean_value(value):
    new_value = ''
    for s in value:
        if s == '.' and '.' in new_value:
            break
        elif s in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '.']:
            new_value += s
        elif s in [' ', '-', '±', '–', ';', 'to', '(', '/', '[']:
            break
    return new_value

In [6]:
# применим функцию к кажому элементу
df['raw_value'] = df['raw_value'].apply(lambda x: clean_value(x))

In [7]:
# переведем колонку raw value в числовой формат 
df['raw_value'] = pd.to_numeric(df['raw_value'], downcast='float')

#### Найдем и удалим выбросы для raw value

In [8]:
df.describe()[['raw_value']]

,raw_value
count,4663.000000
mean,1.959614
std,2.361013
min,0.000000
25%,1.407000
50%,1.494140
75%,1.994500
max,104.000000


In [9]:
fig = px.box(df, y=['raw_value'])
fig.show()

In [9]:
# определим границы выбросов
#q1, q3= np.percentile(df['raw_value'],[25,75])
#iqr = q3 - q1
#lower_bound = q1 - (1.5 * iqr)
#upper_bound = q3 + (1.5 * iqr)

In [10]:
lower_bound = 1
upper_bound = 4

In [11]:
# удалим все что лежит за границами для raw_value
df = df[(df['raw_value'] >= lower_bound) & (df['raw_value'] <= upper_bound)]

In [12]:
# удалим строчки в которых погрещность измерения превосходит медианное значение
df = df[df['measurement_error'] <= df['raw_value'].median()]

In [13]:
df.describe()[['raw_value']]

,raw_value
count,4428.000000
mean,1.731461
std,0.582302
min,1.000000
25%,1.401000
50%,1.488200
75%,1.840500
max,4.000000


#### Найдем медиану для каждого индивидуального вещества из датасета

In [28]:
# найдем количество уникальных smiles
len(pd.unique(df['normalised_name']).tolist())

1334

In [29]:
# сгруппируем по smiles и методу измерения
df_group = df.groupby(['normalised_name', 'measurement_method'], group_keys=True).apply(lambda x: x)

In [30]:
# введем функцию, которая ищет наиболее близкое значение к значениям из листа
def closest_value(input_list, input_value):
 
  arr = np.asarray(input_list)
  i = (np.abs(arr - input_value)).argmin()
  
  return arr[i]

In [31]:
# выберем по одной строке для кажого smiles по принципу близости raw value к медианному значению для этого вещества среди наиболее часто встречающегося метода измерения
df_result = pd.DataFrame()
unique_smiles = pd.unique(df['normalised_name']).tolist()

for smiles in unique_smiles:
    common_metod = df_group.loc[smiles]['measurement_method'].value_counts().idxmax()
    val = closest_value(list(df_group.loc[smiles, common_metod]['raw_value']), df_group.loc[smiles, common_metod]['raw_value'].median())
    row = df[(df['normalised_name'] == smiles) & (df['raw_value'] == val) & (df['measurement_method'] == common_metod)]

    df_result = df_result.append(row.iloc[0])

#### Удалим ненужные (или пока что незаполненные и неиспользуемые) столбцы

In [33]:
df_result_clear = df_result.drop(columns=['DOI', 'Date', 'Journal', 'Title', 'measurement_error',
       'measurement_wavelength', 'measurement_method', 'specifier'])

#### Трансформируем normalised_name в mol файлы для дальшейшего поиска дескрипторов для тех smiles, для которых это возможно

In [ ]:
mols = [Chem.MolFromSmiles(smi) for smi in df_result_clear['normalised_name'].tolist()]
df_result_clear['mol_file'] = mols

In [35]:
df_result_clear.to_csv('6_result.csv', index=False)

In [36]:
df_result_clear.shape

(1334, 4)